# Introduction

The requirements for undergraduate CS certificate can be found at https://guide.wisc.edu/undergraduate/letters-science/computer-sciences/computer-sciences-certificate/#requirementstext. 

# Mathematical Model

Decision variables:

- $x[t, c]$ is a Boolean variable to denote whether to take the class $c$ at semester $t$. 

    - If a student takes CS 200 on his/her first semester, then $x[1, \text{CS 200}] = 1$

    - If a student takes CS 524 on the second Spring semester, then $x[4, \text{CS 524}]= 1$


Prerequisite constraint

- To take CS 300, the student need to take CS 200 in the previous semester, we can encode such prerequisite using the following constraint: 

    $$x[t, \text{CS 300}] \le \sum_{i=1}^{t} x[i, \text{CS 200}] $$


- CS 524 has the prerequisite similar to "(CS 200 or 300) and (MATH 340 or 341)", we can encode it with the constraint:

    $$x[t, \text{CS 524}] \le \sum_{i=1}^{t} x[i, \text{CS 200}] + \sum_{i=1}^{t} x[i, \text{CS 300}] $$

    $$x[t, \text{CS 524}] \le \sum_{i=1}^{t} x[i, \text{CS 340}] + \sum_{i=1}^{t} x[i, \text{CS 341}] $$


Graduation requirement

- We 

# Data Preparation

We first load the json file obtained from the UW-Madison [Course Search & Enrollment website](enroll.wisc.edu), and extract all the CS courses.

In [10]:
using JSON

data = JSON.parsefile("/work/data/1214-spring-2021.json")
data = data["hits"]

cs_cls = Dict(
    Symbol(c["courseDesignation"]) => 
    c for c in data if c["subject"]["shortDescription"] == "COMP SCI"
)

Dict{Symbol,Dict{String,Any}} with 85 entries:
  Symbol("COMP SCI 542") => Dict{String,Any}("honors"=>nothing,"allCrossListedS…
  Symbol("COMP SCI 310") => Dict{String,Any}("honors"=>nothing,"allCrossListedS…
  Symbol("COMP SCI 435") => Dict{String,Any}("honors"=>nothing,"allCrossListedS…
  Symbol("COMP SCI 200") => Dict{String,Any}("honors"=>nothing,"allCrossListedS…
  Symbol("COMP SCI 400") => Dict{String,Any}("honors"=>nothing,"allCrossListedS…
  Symbol("COMP SCI 425") => Dict{String,Any}("honors"=>nothing,"allCrossListedS…
  Symbol("COMP SCI 999") => Dict{String,Any}("honors"=>nothing,"allCrossListedS…
  Symbol("COMP SCI 270") => Dict{String,Any}("honors"=>nothing,"allCrossListedS…
  Symbol("COMP SCI 612") => Dict{String,Any}("honors"=>nothing,"allCrossListedS…
  Symbol("COMP SCI 739") => Dict{String,Any}("honors"=>nothing,"allCrossListedS…
  Symbol("COMP SCI 240") => Dict{String,Any}("honors"=>nothing,"allCrossListedS…
  Symbol("COMP SCI 900") => Dict{String,Any}("honors"=>nothing

For each class, the following information are available to us. We'll use `breadths`, `maximumCredits`, `minimumCredits`, `typicallyOffered`, and `levels` to construct our optimization problem. 

In [11]:
show(IOContext(stdout, :limit => false), "text/plain", cs_cls[Symbol("COMP SCI 524")])

Dict{String,Any} with 44 entries:
  "honors" => nothing
  "allCrossListedSubjects" => Any[Dict{String,Any}("departmentURI"=>"http://www.cs.wisc.edu/","footnotes"=>Any["Courses taught and managed by the Computer Sciences department often have enrollment restrictions that give students in UW-Madison Computer Sciences programs priority access during initial enrollment periods.\n\nEvening exams are likely for most of our undergraduate courses."],"formalDescription"=>"COMPUTER SCIENCES","undergraduateCatalogURI"=>"http://guide.wisc.edu/undergraduate/letters-science/computer-sciences/","termCode"=>"1214","departmentOwnerAcademicOrgCode"=>"L0780","description"=>"COMPUTER SCIENCES","graduateCatalogURI"=>"http://guide.wisc.edu/graduate/computer-sciences/","uddsFundingSource"=>"A4820","shortDescription"=>"COMP SCI","subjectCode"=>"266","schoolCollege"=>Dict{String,Any}("schoolCollegeURI"=>"http://www.ls.wisc.edu/","shortDescription"=>"Letters and Science","formalDescription"=>"Letters and Scienc

In [ ]:
n = length(cs_cls)

85

In [12]:

print(keys(cs_cls))

[Symbol("COMP SCI 542"), Symbol("COMP SCI 310"), Symbol("COMP SCI 435"), Symbol("COMP SCI 200"), Symbol("COMP SCI 400"), Symbol("COMP SCI 425"), Symbol("COMP SCI 999"), Symbol("COMP SCI 270"), Symbol("COMP SCI 612"), Symbol("COMP SCI 739"), Symbol("COMP SCI 240"), Symbol("COMP SCI 900"), Symbol("COMP SCI 475"), Symbol("COMP SCI 899"), Symbol("COMP SCI 567"), Symbol("COMP SCI 252"), Symbol("COMP SCI 537"), Symbol("COMP SCI 570"), Symbol("COMP SCI 638"), Symbol("COMP SCI 682"), Symbol("COMP SCI 990"), Symbol("COMP SCI 532"), Symbol("COMP SCI 220"), Symbol("COMP SCI 298"), Symbol("COMP SCI 579"), Symbol("COMP SCI 369"), Symbol("COMP SCI 525"), Symbol("COMP SCI 540"), Symbol("COMP SCI 679"), Symbol("COMP SCI 704"), Symbol("COMP SCI 790"), Symbol("COMP SCI 352"), Symbol("COMP SCI 354"), Symbol("COMP SCI 710"), Symbol("COMP SCI 368"), Symbol("COMP SCI 681"), Symbol("COMP SCI 538"), Symbol("COMP SCI 552"), Symbol("COMP SCI 402"), Symbol("COMP SCI 320"), Symbol("COMP SCI 534"), Symbol("COMP SC

# Constructing the Optimization Problem

In [ ]:
using JuMP, Cbc

m = Model(Cbc.Optimizer)
n_cls = 10
n_sem = 4

@variable(m, cls[1:n_cls, 1:n_sem], Bin)

@objective(m, Max, sum(cls[i] for i in 1:n))


optimize!(m)
println(termination_status(m))
value.(cls)

OPTIMAL
Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 10 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -10 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
MixedIntegerRounding2 was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
FlowCover was tried 0 times a

10-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=92cfa9df-30ab-40f7-b5d5-752a2fe43860' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>